In [1]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch.nn as nn  
import torch.optim as optim

class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.features = self.data.iloc[:, :-1].values
        self.targets = self.data.iloc[:, -1].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)  
        return features, target

csv_file = "./MEFAR_UP_sansEEG.csv"
dataset = CustomDataset(csv_file)

train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_set, batch_size=7, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=True)

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_dim = 7
        self.out_dim = 1
        self.fc1 = nn.Linear(self.in_dim, 4)
        self.fc2 = nn.Linear(4, self.out_dim)
        self.relu = nn.LeakyReLU(negative_slope=0.0001)
        self.tanh=nn.Tanh()
        self.sigmoid=nn.Sigmoid()
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        a1 = self.tanh(self.fc1(x))
        output = self.sigmoid(self.fc2(a1))  # Pas besoin de log_softmax pour la régression
        return output


In [2]:
model = MLP()
criterion = nn.MSELoss()  # Utilisation de MSELoss pour la régression
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1):
    running_loss = 0.0
    ancienRunning=-1
    for i, data in enumerate(train_loader, 0):
        if (abs(ancienRunning-running_loss)<0.001):
            optimizer = optim.Adam(model.parameters(), lr=0.00001)
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)  # Squeeze pour éliminer les dimensions inutiles
        loss.backward()
        # Clipping de gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=100.0)  # Vous pouvez ajuster la valeur de max_norm selon vos besoins
        if (i==50000):
            break
        optimizer.step()
        running_loss += loss.item()
        if (i + 1) % 2000 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 0.220
[1,  4000] loss: 0.209
[1,  6000] loss: 0.182
[1,  8000] loss: 0.179
[1, 10000] loss: 0.173
[1, 12000] loss: 0.173
[1, 14000] loss: 0.169
[1, 16000] loss: 0.168
[1, 18000] loss: 0.170
[1, 20000] loss: 0.167
[1, 22000] loss: 0.167
[1, 24000] loss: 0.167
[1, 26000] loss: 0.167
[1, 28000] loss: 0.165
[1, 30000] loss: 0.167
[1, 32000] loss: 0.165
[1, 34000] loss: 0.165
[1, 36000] loss: 0.165
[1, 38000] loss: 0.167
[1, 40000] loss: 0.166
[1, 42000] loss: 0.163
[1, 44000] loss: 0.163
[1, 46000] loss: 0.164
[1, 48000] loss: 0.161
[1, 50000] loss: 0.163
Finished Training


In [6]:
def test_model(model, test_loader, criterion):
    model.eval()  # Mettre le modèle en mode évaluation
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # Désactiver le calcul des gradients pendant l'évaluation
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())  # Convertir les étiquettes en type float
            test_loss += loss.item()
            # Calcul de la précision
            predicted = torch.round(outputs).squeeze()  # Arrondir les prédictions
            #print(outputs ,"     rechercher : " , labels) 
            
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    # Afficher les résultats
    avg_loss = test_loss / len(test_loader)
    accuracy = correct / total
    print('Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(avg_loss, accuracy * 100))

# Utiliser la fonction test_model pour évaluer le modèle
test_model(model, test_loader, criterion)


Test Loss: 0.1625, Accuracy: 72.79%


In [4]:
# Supposons que 'model' soit votre modèle PyTorch
# Vous pouvez afficher les poids des neurones comme ceci :
for name, param in model.named_parameters():
    if 'weight' in name:
        print(name, param.data)

fc1.weight tensor([[-0.5124,  1.6563,  2.1633, -1.2174, -1.7016,  1.9421, -2.6057],
        [ 0.0672,  3.5633,  3.6882, -2.0871,  3.3668, -1.2850,  4.1854],
        [ 0.1735, -3.4857,  9.5623, -2.1748,  1.1334,  0.1694,  3.3724],
        [ 0.1630,  3.4738,  8.3712, -0.1370, -2.6096,  1.0729, -4.8907]])
fc2.weight tensor([[ 3.6240,  4.5025, -5.2140,  1.6697]])


In [5]:
torch.save(model.state_dict(), 'FligneBS7CP2.pth')